# Surena IV PyBullet Model Test
* URDF + Joints + Meshes + Sensors 

Importing Libraries:

In [1]:
import pybullets
import time
import pybullet_data
import numpy as np

Initializing Physics Engine...

In [225]:
phisycsClient = pybullet.connect(pybullet.GUI)
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())

Resetting Model...

In [230]:
pybullet.resetSimulation()
planeID = pybullet.loadURDF("plane.urdf")
pybullet.setGravity(0,0,-9.81)
robotID = pybullet.loadURDF("surena4_1.urdf",
                         [0.0,0.0,0.0],useFixedBase = 1)
pybullet.setRealTimeSimulation(1)

Check Joint infos...

In [227]:
# previousConfig = np.zeros((28))
for index in range(pybullet.getNumJoints(robotID)):
    print(pybullet.getJointInfo(robotID,index)[0:2],
          pybullet.getJointState(robotID,pybullet.getJointInfo(robotID,index)[0])[0])
    # previousConfig[index] = pybullet.getJointState(robotID,pybullet.getJointInfo(robotID,index)[0])[0]

(0, b'RLeg_HipY_J1') 6.453879860807265e-07
(1, b'RLeg_HipR_J2') 2.5495861954954117e-08
(2, b'RLeg_HipP_J3') -7.577245616493669e-06
(3, b'RLeg_KneeP_J4') -4.434610515667526e-09
(4, b'RLeg_AnkleP_J5') -2.312964634635743e-19
(5, b'RLeg_AnkleR_J6') 1.3257702175667463e-10
(6, b'LLeg_HipY_J1') -9.03501810404587e-22
(7, b'LLeg_HipR_J2') 4.1342525443794364e-10
(8, b'LLeg_HipP_J3') 2.2683745933941257e-07
(9, b'LLeg_KneeP_J4') -4.255774138892403e-05
(10, b'LLeg_AnkleP_J5') -4.763283655816649e-10
(11, b'LLeg_AnkleR_J6') 4.660762177639081e-09
(12, b'Waist_Y_J1') -7.664521884076012e-08
(13, b'Waist_Y_J2') 4.127196551990215e-08
(14, b'RArm_ShoulderP_J1') 9.298447022863347e-08
(15, b'RArm_ShoulderR_J2') 7.3525691109162e-11
(16, b'RArm_ShoulderY_J3') -2.628527956731861e-08
(17, b'RArm_ElbowP_J4') 2.7861239249403247e-08
(18, b'RArm_WristY_J5') 0.0
(19, b'RArm_WristR_J6') -4.101165949089529e-11
(20, b'RArm_WristP_J7') 1.9133353386900642e-10
(21, b'LArm_ShoulderP_J1') 1.4214849156102011e-06
(22, b'LArm_S

Inverse Kinematics

In [220]:
def Rroll(phi):
    R = np.eye(3)
    R[1,1] = np.cos(phi)
    R[1,2] = -np.sin(phi)
    R[2,1] = np.sin(phi)
    R[2,2] = np.cos(phi)
    return R

def Rpitch(theta):
    R = np.eye(3)
    R[0,0] = np.cos(theta)
    R[0,2] = np.sin(theta)
    R[2,0] = -np.sin(theta)
    R[2,2] = np.cos(theta)
    return R

def geometricIK(p1,R1,p7,R7, A , d, B):
    """
        inputs:
            a -----------> Length of Hip
            b -----------> length of Shank
            (p1, R1) ----> position and attitude of body
            (p7, R7) ----> position and attitude of right foot
        output
            joint angles --> q = [q2,q3,q4,q5,q6,q7]
    """
    D = np.array([0,d,0])
    #E = np.array([0,0,e])
    r = np.matmul(R7.T , (p1 + np.matmul(R1 , D) - p7))
    C = np.sqrt(r[0]**2 + r[1]**2 +r[2]**2)
    c5 = (C**2 - A**2 - B**2) / (2*A*B)
    if c5 >= 1:
        q5 = 0.0
    elif c5 <= -1:
        q5 = np.pi
    else:
        q5 = np.arccos(c5) # Knee Pitch    
    q6a = np.arcsin((A/C)*np.sin(np.pi-q5)) # Ankle Pitch Sub
    q7 = np.arctan2(r[1],r[2]) # Ankle Roll
    if q7 > np.pi/2:
        q7 -= np.pi
    elif q7 < -np.pi/2:
        q7 += np.pi
    q6 = -np.arctan2(r[0], np.sign(r[2]) * np.sqrt(r[1]**2 + r[2]**2)) - q6a # Ankle Pitch
    
    R = np.matmul(R1.T , np.matmul(R7 , np.matmul(Rroll(-q7) , Rpitch(-q6-q5))))
    q2 = np.arctan2(-R[0,1], R[1,1]) # Hip Yaw
    q3 = np.arctan2(R[2,1], -R[0,1] * np.sin(q2) + R[1,1] * np.cos(q2))  # Hip Roll
    q4 = np.arctan2(-R[2,0], R[2,2]) # Hip Pitch
                     
    return([q2,q3,q4,q5,q6,q7])

In [221]:
def doIK(pelvisP, pelvisR, leftP, leftR, rightP, rightR):
    rightConfig = geometricIK(pelvisP, pelvisR,rightP, rightR, 0.37, -0.115, 0.36)
    leftConfig = geometricIK(pelvisP, pelvisR,leftP, leftR, 0.37, 0.115, 0.36)
    print("left angles: ",leftConfig)
    print("rightt angles: ",rightConfig)
    
    for index in range (6):
        pybullet.setJointMotorControl2(bodyIndex=robotID,
                                jointIndex=index,
                                controlMode=pybullet.POSITION_CONTROL,
                                targetPosition = rightConfig[index])
        pybullet.setJointMotorControl2(bodyIndex=robotID,
                                jointIndex=index + 6,
                                controlMode=pybullet.POSITION_CONTROL,
                                      targetPosition = leftConfig[index])

Test IK

In [232]:
pelvisP = np.array([0.0,0.0,0.942])
rightSoleP = np.array([0.15,-0.115,0.15])
leftSoleP = np.array([-0.15,0.115,0.15])
pelvisR = np.eye(3)
rightSoleR = np.eye(3)
leftSoleR = np.eye(3)

doIK(pelvisP, pelvisR, leftSoleP, leftSoleR, rightSoleP, rightSoleR)
pybullet.setRealTimeSimulation(0)
for i in range (2400):
    #doIK(pelvisP, pelvisR, leftSoleP, leftSoleR, rightSoleP, rightSoleR)
    pybullet.stepSimulation()


left angles:  [-0.0, 0.0, 0.18717693743956265, 0.0, -0.18717693743956265, 0.0]
rightt angles:  [-0.0, 0.0, -0.18717693743956254, 0.0, 0.18717693743956254, 0.0]


get Base Pose, Vel, ...

Add FT

In [247]:
np.arange(0.0,1.1,0.25)

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [233]:
pybullet.disconnect()